In [13]:
from tensorflow.keras.models import load_model
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

# Load the model
cnn_model = load_model('cnn_model')
encoder = load_model('encoder_model')


In [14]:
# Preprocessing

base_dir = "./dataset_5/"
categories = ["covid", "normal"]
img_height = 256
img_width = 256
img_paths=[]
def load_images(base_dir, categories):
    images = []
    labels = []

    for category in categories:
        path = os.path.join(base_dir, category)
        class_num = categories.index(category)  # Assign an integer label to each category

        # Iterate over each image in the category directory
        for img in os.listdir(path):
            try:
                # Load the image from the disk
                img_array = cv2.imread(os.path.join(path, img))
                
                # Resize the image
                img_resized = cv2.resize(img_array, (img_height, img_width))

                # Normalize pixel values
                img_resized = img_resized / 255.0

                images.append(img_resized)
                labels.append(class_num)
            except Exception as e:
                print(f"Error: {e}")

    return np.array(images), np.array(labels)

images, labels = load_images(base_dir, categories)
# Encoding labels: 'COVID' to 0 and 'Normal' to 1
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels)


In [15]:
encoded_images = encoder.predict(images)
x_train_encoded, x_test_encoded, y_train_encoded, y_test_encoded = train_test_split(
    encoded_images, 
    labels,
    test_size=0.2, 
    random_state= 123,
    stratify = labels)


14/14 [==============================] - 11s 834ms/step


In [16]:
def report(x_test_encoded, y_test_encoded, model):
    # Perform predictions on the test set
    y_pred = model.predict(x_test_encoded)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test_encoded, axis=1) 

    # Print classification report
    print(classification_report(y_test_classes, y_pred_classes, target_names=categories))

report(x_test_encoded, y_test_encoded, cnn_model)


def auc_roc(x_test_encoded, model):
    # Use the model to predict the probabilities for the test set
    y_prob = model.predict(x_test_encoded)

    # Compute the AUC-ROC
    roc_auc = roc_auc_score(y_test_encoded, y_prob)

    print('AUC-ROC:', roc_auc)

auc_roc(x_test_encoded, cnn_model)


3/3 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

       covid       0.79      0.83      0.81        36
      normal       0.88      0.84      0.86        50

    accuracy                           0.84        86
   macro avg       0.83      0.84      0.83        86
weighted avg       0.84      0.84      0.84        86

3/3 [==============================] - 0s 6ms/step
AUC-ROC: 0.9227777777777778
